In [2]:
from dash import Dash, dcc, html, dash_table, Input, Output, State, callback
import base64
import datetime
import io
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            html.Div('Drag and Drop or', style={'display': 'inline-block', 'verticalAlign': 'middle'}),
            html.Div('t', style={'display': 'inline-block', 'color': 'white'}),  # Adding a white space
            html.A('Select Files', style={'display': 'inline-block', 'verticalAlign': 'middle'})
        ], style={'textAlign': 'center'}),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px auto'  # Center the upload box horizontally
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-data-upload'),
])

def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
            # Categorize purchases based on the 'Category' column
            purchase_categories = df.groupby('Category')['Amount'].sum().reset_index()
            # Filter out payments and credits from purchase categories
            purchase_categories = purchase_categories[purchase_categories['Category'] != 'Payments and Credits']
            # Identify repeat purchases by finding duplicates in the 'Description' column
            repeat_purchases = df[df.duplicated(subset='Description')]
            # Filter out payments and credits from repeat purchases
            repeat_purchases = repeat_purchases[repeat_purchases['Category'] != 'Payments and Credits']
            # Sort repeat purchases by description
            repeat_purchases = repeat_purchases.sort_values(by='Description')
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
            purchase_categories = df.groupby('Category')['Amount'].sum().reset_index()
            purchase_categories = purchase_categories[purchase_categories['Category'] != 'Payments and Credits']
            repeat_purchases = df[df.duplicated(subset='Description')]
            repeat_purchases = repeat_purchases[repeat_purchases['Category'] != 'Payments and Credits']
            repeat_purchases = repeat_purchases.sort_values(by='Description')
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),

        dash_table.DataTable(
            data=purchase_categories.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in purchase_categories.columns],
            style_table={'overflowX': 'auto'},
            style_cell={
                'whiteSpace': 'normal',
                'textAlign': 'left',
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ]
        ),
        
        html.Hr(),  # horizontal line
        
        html.H5('Repeat Purchases:'),
        
        dash_table.DataTable(
            data=repeat_purchases.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in repeat_purchases.columns],
            style_table={'overflowX': 'auto'},
            style_cell={
                'whiteSpace': 'normal',
                'textAlign': 'left',
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ]
        ),

        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        # html.Div('Raw Content'),
        # html.Pre(contents[0:200] + '...', style={
        #     'whiteSpace': 'pre-wrap',
        #     'wordBreak': 'break-all'
        # })
    ])

@app.callback(Output('output-data-upload', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children

if __name__ == '__main__':
    app.run_server(jupyter_mode='tab', debug=True)

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>